 #  <div style="text-align: center"> Santander ML  Explainability  </div> 
###  <div style="text-align: center">CLEAR DATA. MADE MODEL. </div> 
<img src='https://storage.googleapis.com/kaggle-media/competitions/santander/atm_image.png' width=400 height=400>
<div style="text-align:center"> last update: <b>16/02/2019</b></div>



You can Fork code  and  Follow me on:

> ###### [ GitHub](https://github.com/mjbahmani/10-steps-to-become-a-data-scientist)
> ###### [Kaggle](https://www.kaggle.com/mjbahmani/)
-------------------------------------------------------------------------------------------------------------
 <b>I hope you find this kernel helpful and some <font color='red'>UPVOTES</font> would be very much appreciated.</b>
    
 -----------

 <a id="top"></a> <br>
## Notebook  Content
1. [Introduction](#1)
1. [Load packages](#2)
    1. [import](21)
    1. [Setup](22)
    1. [Version](23)
1. [Problem Definition](#3)
    1. [Problem Feature](#31)
    1. [Aim](#32)
    1. [Variables](#33)
    1. [Evaluation](#34)
1. [Exploratory Data Analysis(EDA)](#4)
    1. [Data Collection](#41)
    1. [Visualization](#42)
    1. [Data Preprocessing](#43)
1. [Model Development](#5)
1. [Conclusion](#5)
1. [References](#6)

 <a id="1"></a> <br>
## 1- Introduction
At [Santander](https://www.santanderbank.com) their mission is to help people and businesses prosper. they are always looking for ways to help our customers understand their financial health and identify which products and services might help them achieve their monetary goals.
<img src='https://www.smava.de/kredit/wp-content/uploads/2015/12/santander-bank.png' width=400 height=400>

For anyone taking first steps in data science, **Prediction** is a must know concept. 

 <a id="2"></a> <br>
 ## 2- Load packages
  <a id="21"></a> <br>
## 2-1 Import

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
import matplotlib.patches as patch
import matplotlib.pyplot as plt
from sklearn.svm import NuSVR
from scipy.stats import norm
from sklearn import svm
import lightgbm as lgb
import xgboost as xgb
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import glob
import sys
import os
import gc

 <a id="22"></a> <br>
##  2-2 Setup

In [ ]:
%matplotlib inline
%precision 4
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
np.set_printoptions(suppress=True)
pd.set_option("display.precision", 15)

 <a id="23"></a> <br>
## 2-3 Version


In [ ]:
print('pandas: {}'.format(pd.__version__))
print('numpy: {}'.format(np.__version__))
print('Python: {}'.format(sys.version))

<a id="3"></a> 
<br>
## 3- Problem Definition
In this **challenge**, we should help this **bank**  identify which **customers** will make a **specific transaction** in the future, irrespective of the amount of money transacted. The data provided for this competition has the same structure as the real data we have available to solve this **problem**.


<a id="31"></a> 
### 3-1 Problem Feature

1. train.csv - the training set.
1. test.csv - the test set. The test set contains some rows which are not included in scoring.
1. sample_submission.csv - a sample submission file in the correct format.


<a id="32"></a> 
### 3-2 Aim
In this competition, The task is to predict the value of **target** column in the test set.

<a id="33"></a> 
### 3-3 Variables

We are provided with an **anonymized dataset containing numeric feature variables**, the binary **target** column, and a string **ID_code** column.

The task is to predict the value of **target column** in the test set.


<a id="34"></a> 
## 3-4 evaluation
**Submissions** are evaluated on area under the [ROC curve](http://en.wikipedia.org/wiki/Receiver_operating_characteristic) between the predicted probability and the observed target.
<img src='https://upload.wikimedia.org/wikipedia/commons/6/6b/Roccurves.png' width=300 height=300>

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

<a id="4"></a> 
## 4- Exploratory Data Analysis(EDA)
 In this section, we'll analysis how to use graphical and numerical techniques to begin uncovering the structure of your data. 
*  Data Collection
*  Visualization
*  Data Preprocessing
*  Data Cleaning

 <a id="41"></a> <br>
## 4-1 Data Collection

In [ ]:
print(os.listdir("../input/"))

In [ ]:
# import Dataset to play with it
train= pd.read_csv("../input/train.csv")
test = pd.read_csv('../input/test.csv')

In [ ]:
sample_submission = pd.read_csv('../input/sample_submission.csv')
sample_submission.head()

In [ ]:
train.shape, test.shape, sample_submission.shape

In [ ]:
train.head(5)

##  Data set fields

In [ ]:
train.columns

In [ ]:
print(len(train.columns))

In [ ]:
print(train.info())

## numerical values Describe

In [ ]:
train.describe()

 <a id="42"></a> <br>
## 4-2 Visualization

<a id="421"></a> 
### 4-2-1 hist

In [ ]:
train['target'].value_counts().plot.bar();

In [ ]:
f,ax=plt.subplots(1,2,figsize=(20,10))
train[train['target']==0].var_0.plot.hist(ax=ax[0],bins=20,edgecolor='black',color='red')
ax[0].set_title('target= 0')
x1=list(range(0,85,5))
ax[0].set_xticks(x1)
train[train['target']==1].var_0.plot.hist(ax=ax[1],color='green',bins=20,edgecolor='black')
ax[1].set_title('target= 1')
x2=list(range(0,85,5))
ax[1].set_xticks(x2)
plt.show()

## Mean Frequency

In [ ]:
train[train.columns[2:]].mean().plot('hist');plt.title('Mean Frequency');

<a id="423"></a> 
### 4-2-3 countplot

In [ ]:
f,ax=plt.subplots(1,2,figsize=(18,8))
train['target'].value_counts().plot.pie(explode=[0,0.1],autopct='%1.1f%%',ax=ax[0],shadow=True)
ax[0].set_title('target')
ax[0].set_ylabel('')
sns.countplot('target',data=train,ax=ax[1])
ax[1].set_title('target')
plt.show()

<a id="424"></a> 
### 4-2-4 hist
if you check histogram for all feature, you will find that most of them are so similar

In [ ]:
train["var_0"].hist();

In [ ]:
train["var_81"].hist();

In [ ]:
train["var_2"].hist();

<a id="426"></a> 
### 4-2-6 distplot
 the target in data set is **imbalance**

In [ ]:
sns.set(rc={'figure.figsize':(9,7)})
sns.distplot(train['target']);

<a id="427"></a> 
### 4-2-7 violinplot

In [ ]:
sns.violinplot(data=train,x="target", y="var_0")

In [ ]:
sns.violinplot(data=train,x="target", y="var_81")

 <a id="43"></a> <br>
## 4-3 Data Preprocessing
Before we start this section let me intrduce you, some other compitation that they were similar to this:

1. https://www.kaggle.com/artgor/how-to-not-overfit
1. https://www.kaggle.com/c/home-credit-default-risk
1. https://www.kaggle.com/c/porto-seguro-safe-driver-prediction

## 4-3-1 check missing data for test & train

In [ ]:
def check_missing_data(df):
    flag=df.isna().sum().any()
    if flag==True:
        total = df.isnull().sum()
        percent = (df.isnull().sum())/(df.isnull().count()*100)
        output = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
        data_type = []
        # written by MJ Bahmani
        for col in df.columns:
            dtype = str(df[col].dtype)
            data_type.append(dtype)
        output['Types'] = data_type
        return(np.transpose(output))
    else:
        return(False)

In [ ]:
check_missing_data(train)

In [ ]:
check_missing_data(test)

## 4-3-2 Binary Classification

In [ ]:
train['target'].unique()

## 4-3-3 dataset is imbalanced. 

In [ ]:
train['target'].value_counts()

In [ ]:
def check_balance(df,target):
    check=[]
    # written by MJ Bahmani for binary target
    print('size of data is:',df.shape[0] )
    for i in [0,1]:
        print('for target  {} ='.format(i))
        print(df[target].value_counts()[i]/df.shape[0]*100,'%')
    

In [ ]:
check_balance(train,'target')

## 4-3-4 skewness and kurtosis

In [ ]:
#skewness and kurtosis
print("Skewness: %f" % train['target'].skew())
print("Kurtosis: %f" % train['target'].kurt())

## 4-3-5 feature importance
In this section, I have used this [tutorials](https://www.kaggle.com/dansbecker/permutation-importance), that is amazing for **Permutation Importance**

In [ ]:
cols=["target","ID_code"]
X = train.drop(cols,axis=1)
y = train["target"]


In [ ]:
X_test  = test.drop("ID_code",axis=1)

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
rfc_model = RandomForestClassifier(random_state=0).fit(train_X, train_y)

Here is how to calculate and show importances with the [eli5](https://eli5.readthedocs.io/en/latest/) library:

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(rfc_model, random_state=1).fit(val_X, val_y)
eli5.show_weights(perm, feature_names = val_X.columns.tolist())

## 4-3-5-1 Interpreting Permutation Importances
1. The values towards the top are the most important features, and those towards the bottom matter least.

1. The first number in each row shows how much model performance decreased with a random shuffling (in this case, using "accuracy" as the performance metric).

1. Like most things in data science, there is some randomness to the exact performance change from a shuffling a column. We measure the amount of randomness in our permutation importance calculation by repeating the process with multiple shuffles. The number after the ± measures how performance varied from one-reshuffling to the next. [1]

**Given the above, can it  be concluded that almost all the features are equally important?!**

## 4-3-5-2 Partial Dependence Plots
While feature importance shows what variables most affect predictions, partial dependence plots show how a feature affects predictions.[1]

In [ ]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state=1)
tree_model = DecisionTreeClassifier(random_state=0, max_depth=5, min_samples_split=5).fit(train_X, train_y)

For the sake of explanation, I use a Decision Tree which you can see below.



In [ ]:
features = [c for c in train.columns if c not in ['ID_code', 'target']]

In [ ]:
from sklearn import tree
import graphviz
tree_graph = tree.export_graphviz(tree_model, out_file=None, feature_names=features)
graphviz.Source(tree_graph)

As guidance to read the tree:

1. Leaves with children show their splitting criterion on the top
1. The pair of values at the bottom show the count of True values and False values for the target respectively, of data points in that node of the tree.

In [ ]:
from matplotlib import pyplot as plt
from pdpbox import pdp, get_dataset, info_plots

# Create the data that we will plot
pdp_goals = pdp.pdp_isolate(model=tree_model, dataset=val_X, model_features=features, feature='var_81')

# plot it
pdp.pdp_plot(pdp_goals, 'var_81')
plt.show()

A few items are worth pointing out as you interpret this plot

1. The y axis is interpreted as change in the prediction from what it would be predicted at the baseline or leftmost value.
1. A blue shaded area indicates level of confidence
1. From this particular graph, we see that scoring a goal substantially increases your chances of winning "Player of The Game." But extra goals beyond that appear to have little impact on predictions.

 <a id="5"></a> <br>
# 5- sample model
So far, we have used two  models, and at this point we add another model and we'll be expanding it soon.

In [ ]:
Y_pred_rfc = rfc_model.predict(X_test)

or we can use tree_model

In [ ]:
Y_pred_tree = tree_model.predict(X_test)

You can change your model and submit the results of other models

In [ ]:
submission1_xgboost = pd.DataFrame({
        "ID_code": test["ID_code"],
        "target": Y_pred_rfc
    })
submission1_xgboost.to_csv('submission.csv', index=False)

you can follow me on:
> ###### [ GitHub](https://github.com/mjbahmani/)
> ###### [Kaggle](https://www.kaggle.com/mjbahmani/)

 <b>I hope you find this kernel helpful and some <font color='red'>UPVOTES</font> would be very much appreciated.<b/>
 

# References & credits

1. [permutation-importance](https://www.kaggle.com/dansbecker/permutation-importance)
1. [partial-plots](https://www.kaggle.com/dansbecker/partial-plots)

Go to first step: [Course Home Page](https://www.kaggle.com/mjbahmani/10-steps-to-become-a-data-scientist)

Go to next step : [Titanic](https://www.kaggle.com/mjbahmani/a-comprehensive-ml-workflow-with-python)


# Not Completed yet!!!